In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn import preprocessing
import sklearn.metrics as metrics
import numpy as np
import copy


C:\Users\Bhavya\Anaconda3\envs\AppliedDataScience\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [17]:
def stepwise_action_window_creator(df,window_size,step_size):
    dflist = []
    index = df.index.values
    for i in range(0,len(index),step_size):
        if(i > len(index) - window_size):
            break;
        else:
            dflist.append(df.iloc[i:i+window_size])   
    for i in range(0,len(dflist)):
        index = dflist[i].index.values
        dflist[i] = dflist[i].assign(time = range(index[-1]+1 - index[0]))
        dflist[i] = dflist[i].assign(act_id = [i]*len(dflist[i]))
    result = pd.concat(dflist)
    return result

In [18]:
sub1_pushing = pd.read_csv('sub1_Pushing_processed.txt', sep="\s+", header=None)
sub1_pushing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub1_pushing = stepwise_action_window_creator(sub1_pushing,1000,100)

sub2_pushing = pd.read_csv('sub2_Pushing_processed.txt', sep="\s+", header=None)
sub2_pushing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub2_pushing = stepwise_action_window_creator(sub2_pushing,1000,100)

sub3_pushing = pd.read_csv('sub3_Pushing_processed.txt', sep="\s+", header=None)
sub3_pushing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub3_pushing = stepwise_action_window_creator(sub3_pushing,1000,100)

sub4_pushing = pd.read_csv('sub4_Pushing_processed.txt', sep="\s+", header=None)
sub4_pushing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub4_pushing = stepwise_action_window_creator(sub4_pushing,1000,100)

In [23]:
sub1_sidekicking = pd.read_csv('sub1_Sidekicking_processed.txt', sep="\s+", header=None)
sub1_sidekicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub1_sidekicking = stepwise_action_window_creator(sub1_sidekicking,1000,100)

sub2_sidekicking = pd.read_csv('sub2_Sidekicking_processed.txt', sep="\s+", header=None)
sub2_sidekicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub2_sidekicking = stepwise_action_window_creator(sub2_sidekicking,1000,100)

sub3_sidekicking = pd.read_csv('sub3_Sidekicking_processed.txt', sep="\s+", header=None)
sub3_sidekicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub3_sidekicking = stepwise_action_window_creator(sub3_sidekicking,1000,100)

sub4_sidekicking = pd.read_csv('sub4_Sidekicking_processed.txt', sep="\s+", header=None)
sub4_sidekicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub4_sidekicking = stepwise_action_window_creator(sub4_sidekicking,1000,100)


In [38]:
sub1_slapping = pd.read_csv('sub1_Slapping_processed.txt', sep="\s+", header=None)
sub1_slapping.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub1_slapping = stepwise_action_window_creator(sub1_slapping,1000,100)

sub2_slapping = pd.read_csv('sub2_Slapping_processed.txt', sep="\s+", header=None)
sub2_slapping.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub2_slapping = stepwise_action_window_creator(sub2_slapping,1000,100)

sub3_slapping = pd.read_csv('sub3_Slapping_processed.txt', sep="\s+", header=None)
sub3_slapping.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub3_slapping = stepwise_action_window_creator(sub3_slapping,1000,100)

sub4_slapping = pd.read_csv('sub4_Slapping_processed.txt', sep="\s+", header=None)
sub4_slapping.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub4_slapping = stepwise_action_window_creator(sub4_slapping,1000,100)


In [39]:
sub1_slapping.head()

,R-Bic,R-Tri,L-Bic,L-Tri,R-Thi,R-Ham,L-Thi,L-Ham,time,act_id
0,768.939613,-2.016984,200.258532,53.844564,2882.679957,548.971366,289.435435,1363.487393,0,0
1,1077.817107,58.373533,163.576423,59.016179,2969.659601,519.288940,449.296042,1302.010863,1,0
2,1354.117022,114.957874,132.614464,64.062779,3043.380727,491.098961,601.876909,1244.532826,2,0
3,1599.317028,167.839294,107.095724,68.985790,3104.593844,464.358739,747.227875,1190.889247,3,0
4,1814.894798,217.121052,86.743273,73.786639,3154.049462,439.025584,885.398781,1140.916086,4,0


In [45]:
sub1_frontkicking = pd.read_csv('sub1_Frontkicking_processed.txt', sep="\s+", header=None)
sub1_frontkicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub1_frontkicking = stepwise_action_window_creator(sub1_frontkicking,1000,100)

sub2_frontkicking = pd.read_csv('sub2_Frontkicking_processed.txt', sep="\s+", header=None)
sub2_frontkicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub2_frontkicking = stepwise_action_window_creator(sub2_frontkicking,1000,100)

sub3_frontkicking = pd.read_csv('sub3_Frontkicking_processed.txt', sep="\s+", header=None)
sub3_frontkicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub3_frontkicking = stepwise_action_window_creator(sub3_frontkicking,1000,100)

sub4_frontkicking = pd.read_csv('sub4_Frontkicking_processed.txt', sep="\s+", header=None)
sub4_frontkicking.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub4_frontkicking = stepwise_action_window_creator(sub4_frontkicking,1000,100)


In [46]:
sub1_elbowing = pd.read_csv('sub1_Elbowing_processed.txt', sep="\s+", header=None)
sub1_elbowing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub1_elbowing = stepwise_action_window_creator(sub1_elbowing,1000,100)

sub2_elbowing = pd.read_csv('sub2_Elbowing_processed.txt', sep="\s+", header=None)
sub2_elbowing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub2_elbowing = stepwise_action_window_creator(sub2_elbowing,1000,100)

sub3_elbowing = pd.read_csv('sub3_Elbowing_processed.txt', sep="\s+", header=None)
sub3_elbowing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub3_elbowing = stepwise_action_window_creator(sub3_elbowing,1000,100)

sub4_elbowing = pd.read_csv('sub4_Elbowing_processed.txt', sep="\s+", header=None)
sub4_elbowing.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub4_elbowing = stepwise_action_window_creator(sub4_elbowing,1000,100)


In [47]:
sub1_pulling = pd.read_csv('sub1_Pulling_processed.txt', sep="\s+", header=None)
sub1_pulling.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub1_pulling = stepwise_action_window_creator(sub1_pulling,1000,100)

sub2_pulling = pd.read_csv('sub2_Pulling_processed.txt', sep="\s+", header=None)
sub2_pulling.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub2_pulling = stepwise_action_window_creator(sub2_pulling,1000,100)

sub3_pulling = pd.read_csv('sub3_Pulling_processed.txt', sep="\s+", header=None)
sub3_pulling.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub3_pulling = stepwise_action_window_creator(sub3_pulling,1000,100)

sub4_pulling = pd.read_csv('sub4_Pulling_processed.txt', sep="\s+", header=None)
sub4_pulling.columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri","R-Thi", "R-Ham", "L-Thi", "L-Ham"]
sub4_pulling = stepwise_action_window_creator(sub4_pulling,1000,100)


In [27]:
extraction_settings = EfficientFCParameters()

In [28]:
sub1_pushing_features = extract_features(sub1_pushing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:16,  9.81s/it]                                                                                                                                                                       


In [29]:
sub2_pushing_features = extract_features(sub2_pushing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:22, 10.15s/it]                                                                                                                                                                       


In [30]:
sub3_pushing_features = extract_features(sub3_pushing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:31<00:00, 10.59s/it]


In [31]:
sub4_pushing_features = extract_features(sub4_pushing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:18,  9.95s/it]                                                                                                                                                                       


In [32]:
sub1_sidekicking_features = extract_features(sub1_sidekicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:25, 10.29s/it]                                                                                                                                                                       


In [33]:
sub2_sidekicking_features = extract_features(sub2_sidekicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:22, 10.14s/it]                                                                                                                                                                       


In [34]:
sub3_sidekicking_features = extract_features(sub3_sidekicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:30, 10.51s/it]                                                                                                                                                                       


In [35]:
sub4_sidekicking_features = extract_features(sub4_sidekicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:51, 11.59s/it]                                                                                                                                                                       


In [40]:
sub1_slapping_features = extract_features(sub1_slapping, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:42, 11.11s/it]                                                                                                                                                                       


In [41]:
sub2_slapping_features = extract_features(sub2_slapping, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:54, 11.73s/it]                                                                                                                                                                       


In [42]:
sub3_slapping_features = extract_features(sub3_slapping, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:59, 12.00s/it]                                                                                                                                                                       


In [43]:
sub4_slapping_features = extract_features(sub4_slapping, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [04:02, 12.13s/it]                                                                                                                                                                       


In [48]:
sub1_frontkicking_features = extract_features(sub1_frontkicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:20, 10.02s/it]                                                                                                                                                                       


In [49]:
sub2_frontkicking_features = extract_features(sub2_frontkicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:22, 10.11s/it]                                                                                                                                                                       


In [50]:
sub3_frontkicking_features = extract_features(sub3_frontkicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:30, 10.52s/it]                                                                                                                                                                       


In [51]:
sub4_frontkicking_features = extract_features(sub4_frontkicking, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:03,  9.18s/it]                                                                                                                                                                       


In [52]:
sub1_elbowing_features = extract_features(sub1_elbowing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:09,  9.46s/it]                                                                                                                                                                       


In [53]:
sub2_elbowing_features = extract_features(sub2_elbowing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:27, 10.38s/it]                                                                                                                                                                       


In [54]:
sub3_elbowing_features = extract_features(sub3_elbowing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:16,  9.85s/it]                                                                                                                                                                       


In [55]:
sub4_elbowing_features = extract_features(sub4_elbowing, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:15,  9.78s/it]                                                                                                                                                                       


In [56]:
sub1_pulling_features = extract_features(sub1_pulling, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:12,  9.63s/it]                                                                                                                                                                       


In [57]:
sub2_pulling_features = extract_features(sub2_pulling, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:25, 10.28s/it]                                                                                                                                                                       


In [58]:
sub3_pulling_features = extract_features(sub3_pulling, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:25, 10.25s/it]                                                                                                                                                                       


In [59]:
sub4_pulling_features = extract_features(sub4_pulling, 
                     column_id='act_id', column_sort='time',n_jobs = 4,
                     default_fc_parameters=extraction_settings,
                     impute_function= impute)

Feature Extraction: 20it [03:25, 10.29s/it]                                                                                                                                                                       


In [62]:

sub1_pushing_features.to_csv('sub1_pushing_processed_features.csv')
sub1_sidekicking_features.to_csv('sub1_sidekicking_processed_features.csv')
sub1_slapping_features.to_csv('sub1_slapping_processed_features.csv')
sub1_frontkicking_features.to_csv('sub1_frontkicking_processed_features.csv')
sub1_elbowing_features.to_csv('sub1_elbowing_processed_features.csv')
sub1_pulling_features.to_csv('sub1_pulling_processed_features.csv')

sub2_pushing_features.to_csv('sub2_pushing_processed_features.csv')
sub2_sidekicking_features.to_csv('sub2_sidekicking_processed_features.csv')
sub2_slapping_features.to_csv('sub2_slapping_processed_features.csv')
sub2_frontkicking_features.to_csv('sub2_frontkicking_processed_features.csv')
sub2_elbowing_features.to_csv('sub2_elbowing_processed_features.csv')
sub2_pulling_features.to_csv('sub2_pulling_processed_features.csv')

sub3_pushing_features.to_csv('sub3_pushing_processed_features.csv')
sub3_sidekicking_features.to_csv('sub3_sidekicking_processed_features.csv')
sub3_slapping_features.to_csv('sub3_slapping_processed_features.csv')
sub3_frontkicking_features.to_csv('sub3_frontkicking_processed_features.csv')
sub3_elbowing_features.to_csv('sub3_elbowing_processed_features.csv')
sub3_pulling_features.to_csv('sub3_pulling_processed_features.csv')

sub4_pushing_features.to_csv('sub4_pushing_processed_features.csv')
sub4_sidekicking_features.to_csv('sub4_sidekicking_processed_features.csv')
sub4_slapping_features.to_csv('sub4_slapping_processed_features.csv')
sub4_frontkicking_features.to_csv('sub4_frontkicking_processed_features.csv')
sub4_elbowing_features.to_csv('sub4_elbowing_processed_features.csv')
sub4_pulling_features.to_csv('sub4_pulling_processed_features.csv')